In [ ]:
pip install snowflake-connector-python snowflake-ingest

In [ ]:
import snowflake.connector
import time
import random
import json
import uuid
import os
from datetime import datetime, timedelta, date

# Connect to Snowflake
conn = snowflake.connector.connect(
    user='Snowflake',
    password='Snowflake123',
    account='XLOABBV-NG32826',
    warehouse='COMPUTE_WH',
    database='TEST5',
    schema='STREAMING'
)

# Function to generate random date within a range
def random_date(start_date, end_date):
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    return start_date + timedelta(days=random_number_of_days)

# Function to generate sample data based on the specified schema
def generate_sample_data(num_records=100):
    genders = ['Male', 'Female']
    locations = ['Colombo', 'Gampaha', 'Kaluthara', 'Galle', 'Jaffna', 
                'Trincomalee', 'Kurunegala', 'Ratnapura', 'Anuradhapura', 'Kandy']
    
    product_mapping = {
        'A001': {'name': 'Lightning Cable', 'price': 1000},
        'A002': {'name': 'Power Adapter20W', 'price': 2500},
        'A003': {'name': 'TypeC Cable', 'price': 800},
        'A004': {'name': 'Power Bank', 'price': 5000},
        'A005': {'name': 'Smart Watch', 'price': 7000}
    }
    
    product_ids = list(product_mapping.keys())
    device_types = ['Mobile Phone', 'Personal Computer']
    purchasing_types = ['Cash On Delivery', 'Card Pay', 'KOKO Pay']
    
    # Date ranges
    stock_start_date = date(2020, 1, 1)
    stock_end_date = date(2022, 12, 31)
    
    purchase_start_date = date(2023, 1, 1)
    purchase_end_date = date(2024, 3, 31)
    
    records = []
    for i in range(num_records):
        product_id = random.choice(product_ids)
        product_info = product_mapping[product_id]
        quantity = random.randint(1, 5)
        
        record = {
            'indexid': i + 1,
            'user_id': f'user_{random.randint(1000, 9999)}',
            'gender': random.choice(genders),
            'location': random.choice(locations),
            'product_id': product_id,
            'product_name': product_info['name'],
            'stock_date': random_date(stock_start_date, stock_end_date).isoformat(),
            'unit_price': product_info['price'],
            'quantity': quantity,
            'total_price': quantity * product_info['price'],
            'device_type': random.choice(device_types),
            'purchasing_type': random.choice(purchasing_types),
            'purchase_date': random_date(purchase_start_date, purchase_end_date).isoformat()
        }
        records.append(record)
    
    return records

# Create cursor for executing SQL
cursor = conn.cursor()

try:
    # Generate data
    records = generate_sample_data(100)  # Generate 1000 records
    
    # Write records to a JSON file
    with open("streaming_data.json", "w") as f:
        for record in records:
            json.dump(record, f)
            f.write("\n")
    
    # Create a temporary stage for file upload
    cursor.execute("CREATE TEMPORARY STAGE IF NOT EXISTS temp_streaming_stage")
    
    # Upload the file to the temporary stage
    cursor.execute("PUT file://streaming_data.json @temp_streaming_stage")
    
    # Copy data from staged file to table
    cursor.execute("""
    COPY INTO raw_data
    FROM @temp_streaming_stage/streaming_data.json.gz
    FILE_FORMAT = (TYPE = 'JSON')
    MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
    """)
    
    print(f"Successfully loaded {len(records)} records into the raw_data table")
    
except Exception as e:
    print(f"Error loading data: {e}")
    
finally:
    # Clean up
    try:
        cursor.execute("DROP STAGE IF EXISTS temp_streaming_stage")
        os.remove("streaming_data.json")
    except Exception as e:
        print(f"Error during cleanup: {e}")
    
    # Close connections
    cursor.close()
    conn.close()
    print("Connections closed")